#영화 리뷰 데이터(NSMC)를 활용한 감성 분석
---

#(1) Step1. 데이터 로드와 구조 확인

In [ ]:
!pip install konlpy

In [ ]:
import pandas as pd

# 데이터셋 다운로드 경로
train_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt"
test_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt"

# 데이터 로드
train_data = pd.read_csv(train_url, sep="\t", encoding="utf-8")
test_data = pd.read_csv(test_url, sep="\t", encoding="utf-8")

# 데이터 확인
print(train_data.head())
print(test_data.head())


#(2) Step2: 데이터 전처리 (결측치 제거)

In [ ]:
# 불필요한 결측치 제거
train_data = train_data.dropna()
test_data = test_data.dropna()

#(3) Step3: 텍스트 데이터 전처리 (정규식, 형태소분석 및 어간추출)

In [ ]:
import re
from konlpy.tag import Okt
okt = Okt()

stopwords = set([
    '영화', '보다', '있다', '이다', '하다','그냥', '정말', '좀', '역시', '이건',
    '너무', '이', '그', '가', '를', '은', '는', '의', '에', '도', '으로', '하고',
    '그리고', '하지만', '더', '또', '에서', '보다도'
])

# 텍스트 전처리 함수
def preprocess_text(text):
    text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", text)  # 한글 및 공백 제외 제거
    tokens = okt.morphs(text, stem=True)  # 형태소 분석 및 어간 추출
    tokens = [word for word in tokens if word not in stopwords]  # 불용어 제거
    return " ".join(tokens)
# 전처리 적용
train_data["cleaned_document"] = train_data["document"].apply(preprocess_text)
test_data["cleaned_document"] = test_data["document"].apply(preprocess_text)
# 전처리 결과 확인
print(train_data[["document", "cleaned_document"]].head())


#(4) Step4: 텍스트 벡터화

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_data["cleaned_document"])
X_test = vectorizer.transform(test_data["cleaned_document"])
# 라벨
y_train = train_data["label"]
y_test = test_data["label"]

print(f"훈련 데이터 TF-IDF 크기: {X_train.shape}")
print(f"테스트 데이터 TF-IDF 크기: {X_test.shape}")


#(5) Step5: 머신러닝 모델 학습

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
# 로지스틱 회귀 모델 학습
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
# 예측 및 성능 평가
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


#(6) Step6: 시각화

In [ ]:
!apt-get update && apt-get install -y fonts-nanum

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
# 긍정/부정 리뷰 분리
positive_reviews = " ".join(train_data[train_data["label"] == 1]["cleaned_document"])
negative_reviews = " ".join(train_data[train_data["label"] == 0]["cleaned_document"])

# WordCloud 생성
wordcloud_positive = WordCloud(font_path=font_path, background_color='white').generate(positive_reviews)
wordcloud_negative = WordCloud(font_path=font_path, background_color='black').generate(negative_reviews)

# 시각화
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Positive Reviews")
plt.imshow(wordcloud_positive, interpolation="bilinear")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Negative Reviews")
plt.imshow(wordcloud_negative, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Positive Reviews")
plt.imshow(wordcloud_positive, interpolation="bilinear")
plt.axis("off")
plt.subplot(1, 2, 2)
plt.title("Negative Reviews")
plt.imshow(wordcloud_negative, interpolation="bilinear")
plt.axis("off")
plt.show()

---
#전체코드


In [ ]:
import pandas as pd
# 데이터셋 다운로드 경로
train_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt"
test_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt"
# 데이터 로드
train_data = pd.read_csv(train_url, sep="\t")
test_data = pd.read_csv(test_url, sep="\t")
# 데이터 확인
print(train_data.head())
print(test_data.head())

train_data = train_data.dropna()
test_data = test_data.dropna()

import re
from konlpy.tag import Okt
okt = Okt()
# 텍스트 전처리 함수
def preprocess_text(text):
    text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", text)  # 한글 및 공백 제외 제거
    tokens = okt.morphs(text, stem=True)  # 형태소 분석 및 어간 추출
    return " ".join(tokens)
# 전처리 적용
train_data["cleaned_document"] = train_data["document"].apply(preprocess_text)
test_data["cleaned_document"] = test_data["document"].apply(preprocess_text)
print(train_data[["document", "cleaned_document"]].head()) #결과 출력

import re
from konlpy.tag import Okt
okt = Okt()

stopwords = set([
    '영화', '보다', '있다', '이다', '그냥', '정말', '좀', '역시', '이건',
    '너무', '이', '그', '가', '를', '은', '는', '의', '에', '도', '으로', '하고',
    '그리고', '하지만', '더', '또', '에서', '보다도'
])

# 텍스트 전처리 함수
def preprocess_text(text):
    text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", text)  # 한글 및 공백 제외 제거
    tokens = okt.morphs(text, stem=True)  # 형태소 분석 및 어간 추출
    tokens = [word for word in tokens if word not in stopwords]  # 불용어 제거
    return " ".join(tokens)
# 전처리 적용
train_data["cleaned_document"] = train_data["document"].apply(preprocess_text)
test_data["cleaned_document"] = test_data["document"].apply(preprocess_text)
# 전처리 결과 확인
print(train_data[["document", "cleaned_document"]].head())

from sklearn.feature_extraction.text import TfidfVectorizer
# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_data["cleaned_document"])
X_test = vectorizer.transform(test_data["cleaned_document"])
# 라벨
y_train = train_data["label"]
y_test = test_data["label"]

print(f"훈련 데이터 TF-IDF 크기: {X_train.shape}")
print(f"테스트 데이터 TF-IDF 크기: {X_test.shape}")

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
# 로지스틱 회귀 모델 학습
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
# 예측 및 성능 평가
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))